# **"SUPPORT 2" DATASET**

#**NORMALIZATION**

# Missing values


In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
from scipy.stats import f_oneway
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

data = pd.read_csv("/content/DataSet_AI2.csv")

#replace missing values with values stated in the dataset website
data['alb'].fillna(3.5, inplace=True)
data['pafi'].fillna(333.3, inplace=True)
data['bili'].fillna(1.01, inplace=True)
data['crea'].fillna(1.01, inplace=True)
data['bun'].fillna(6.51, inplace=True)
data['wblc'].fillna(9, inplace=True)
data['urine'].fillna(2502, inplace=True)

#replace missing values of income column
def extract_numeric_income(income_str):
     if isinstance(income_str, (str, bytes)):
        match = re.search(r'\$?(\d+)', income_str)
        if match:
            return int(match.group(1))
     return None

data['income'] = data['income'].apply(extract_numeric_income)

average_income = data['income'].mean(skipna=True)

data['income'].fillna(average_income, inplace=True)

#replace missing values of scoma column
data['scoma'].fillna(data['scoma'].mean(), inplace=True)

#replace missing values of sps column
data['sps'].fillna(data['sps'].mean(), inplace=True)

#replace missing values of aps column
data['aps'].fillna(data['aps'].mean(), inplace=True)

#replace missing values of race column
mode_value = data['race'].mode()[0]
data['race'].fillna(mode_value, inplace=True)

#replace missing values of surv2m column
data['surv2m'].fillna(data['surv2m'].mean(), inplace=True)

#replace missing values of surv6m column
data['surv6m'].fillna(data['surv6m'].mean(), inplace=True)

#replace missing values of prg2m column
data['prg2m'].fillna(data['prg2m'].mean(), inplace=True)

#replace missing values of prg6m column
data['prg6m'].fillna(data['prg6m'].mean(), inplace=True)

#replace missing values in dnr column
mode_value = data['dnr'].mode()[0]
data['dnr'].fillna(mode_value, inplace=True)

#replace missing values of dnrday column
data['dnrday'].fillna(data['dnrday'].mean(), inplace=True)

#replace missing values of meanbp column
data['meanbp'].fillna(data['meanbp'].mean(), inplace=True)

#replace missing values of hrt column
data['hrt'].fillna(data['hrt'].mean(), inplace=True)

#replace missing values of meanbp column
data['resp'].fillna(data['resp'].mean(), inplace=True)

#replace missing values of temp column
data['temp'].fillna(data['temp'].mean(), inplace=True)

#replace missing values of sod column
data['sod'].fillna(data['sod'].mean(), inplace=True)

#replace missing values of ph column
data['ph'].fillna(data['ph'].mean(), inplace=True)

#replace missing values of glucose column
data['glucose'].fillna(data['glucose'].mean(), inplace=True)

#replace missing values of adlp column
data['adlp'].fillna(data['adlp'].mean(), inplace=True)

#replace missing values of adls column
data['adls'].fillna(data['adls'].mean(), inplace=True)

#replace missing values of totcst column
data['totcst'].fillna(data['totcst'].mean(), inplace=True)

#replace missing values of avtisst column
data['avtisst'].fillna(data['avtisst'].mean(), inplace=True)

#replace missing values of totmcst column
data['totmcst'] = pd.to_numeric(data['totmcst'], errors='coerce')
data['totmcst'].fillna(data['totmcst'].mean(), inplace=True)

#replace missing values of charges column
data['charges'].fillna(data['charges'].mean(), inplace=True)

#replace missing values of edu column
data['edu'].fillna(data['edu'].mean(), inplace=True)


data.to_csv('/content/Dataset_No_Missing_Values.csv', index='False')

# Min-Max normalization

In [ ]:
df = data

# Min-Max Normalization function
def min_max_normalization(data):
    if data.dtype in ['int64', 'float64']:
        min_val = data.min()
        max_val = data.max()
        normalized_data = (data - min_val) / (max_val - min_val)
        return normalized_data
    else:
        return data

# Define columns for Min-Max normalization
columns_min_max = ['age', 'num.co', 'sps', 'aps', 'surv2m', 'surv6m', 'hday', 'prg2m', 'prg6m', 'dnrday', 'meanbp',
                   'wblc', 'hrt', 'resp', 'temp', 'pafi', 'bili', 'crea', 'sod', 'ph', 'glucose', 'adlp', 'adls',
                   'adlsc', 'urine', 'bun', 'income', 'alb']

# Apply Min-Max Normalization
for column in columns_min_max:
    df[column] = min_max_normalization(data[column])


df.to_csv('/content/Processed_DataSet_Min-Max_AI.csv', index=False)
print(" ")
print(" ")
print(" ")


# Z-Score normalization

In [ ]:
print("Z-SCORE NORMALIZATION")

print(" ")

df = data


def z_score_normalization(data):
  if data.dtype in ['int64', 'float64']:
    mean_val = data.mean()
    std_val = data.std()
    normalized_data = (data - mean_val) / std_val
    return normalized_data
  else:
        return data

# Specify the columns to be normalized
columns_to_normalize = ['age', 'num.co', 'sps', 'aps', 'surv2m', 'surv6m', 'hday', 'prg2m', 'prg6m', 'dnrday', 'meanbp',
                         'wblc', 'hrt', 'resp', 'temp', 'pafi', 'bili', 'crea', 'sod', 'ph', 'glucose', 'adlp', 'adls',
                         'adlsc', 'urine', 'bun', 'income', 'alb', 'edu', 'charges', 'totcst', 'totmcst', 'avtisst']

# Z-score normalization
for column in columns_to_normalize:
    df[column] = z_score_normalization(data[column])

# Exclude non-numeric columns
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns

df.to_csv('/content/Processed_DataSet_Z-score_AI_.csv', index=False)



# **TARGET 1 (SFDM2)**

# **SFDM2 MODEL: ENSEMBLE MODEL RANDOM FOREST PLUS FEED FORWARD NEURAL NETWORK**


###MIN-MAX NORMALIZED DATASET

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [56]:
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("/content/Processed_DataSet_Min-Max_AI.csv")
data.dropna(subset=['sfdm2'], inplace=True)
data.drop(['edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'death', 'hospdead','id'],axis=1, inplace=True)

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
data[columns_to_encode] = data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


In [ ]:
target = data['sfdm2']
x = data.iloc[:, :-1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.2, random_state=42)

In [ ]:
# Train a Random Forest classifier on the reduced data
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = clf.predict(X_test)

# Evaluate the accuracy of the Random Forest classifier
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy of Random Forest Classifier: {accuracy_rf:.2f}')
print("        ")

In [ ]:
from keras.activations import sigmoid
from keras.activations import tanh
from keras.layers import LeakyReLU

# Convert target labels to one-hot encoding for neural network
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.to_numpy().reshape(-1, 1))


# Build a simple neural network
model = Sequential()
model.add(Dense(16, input_dim=37, activation='sigmoid'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model Architecture:", model.summary())
print("        ")


model.fit(X_train, y_train_encoded, epochs=50, batch_size=8, verbose=0)

# Evaluate the neural network on the test set
_, accuracy_nn = model.evaluate(X_test, y_test_encoded)
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")


In [ ]:
# Combine predictions from Random Forest and Neural Network (Ensemble)
y_pred_ensemble = np.argmax(model.predict(X_test) + clf.predict_proba(X_test), axis=1)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

In [ ]:
print(f'Accuracy of Random Forest Classifier: {accuracy_rf:.2f}')
print("        ")
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

###Z-Score NORMALIZED DATASET

In [59]:
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("/content/Processed_DataSet_Z-score_AI_.csv")
data.dropna(subset=['sfdm2'], inplace=True)
data.drop(['edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'death', 'hospdead','id'],axis=1, inplace=True)

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
data[columns_to_encode] = data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

In [ ]:
target = data['sfdm2']
x = data.iloc[:, :-1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.4, random_state=42)

In [ ]:
# Train a Random Forest classifier on the reduced data
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = clf.predict(X_test)

# Evaluate the accuracy of the Random Forest classifier
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy of Random Forest Classifier: {accuracy_rf:.2f}')
print("        ")

In [ ]:
# Convert target labels to one-hot encoding for neural network
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.to_numpy().reshape(-1, 1))


# Build a simple neural network
model = Sequential()
model.add(Dense(16, input_dim=37, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model Architecture:", model.summary())
print("        ")


# Train the neural network on the reduced data obtained through PCA
model.fit(X_train, y_train_encoded, epochs=50, batch_size=8, verbose=0)

# Evaluate the neural network on the test set
_, accuracy_nn = model.evaluate(X_test, y_test_encoded)
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")


In [ ]:
# Combine predictions from Random Forest and Neural Network (Ensemble)
y_pred_ensemble = np.argmax(model.predict(X_test) + clf.predict_proba(X_test), axis=1)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

97/97 [==============================] - 0s 1ms/step
Accuracy of Ensemble Model: 0.72


In [ ]:
print(f'Accuracy of Random Forest Classifier: {accuracy_rf:.2f}')
print("        ")
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

# **SFDM2 MODEL: HYBRID WITH LOGICAL REGRESSION**

###MIN-MAX NORMALIZED DATASET

In [61]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.preprocessing import StandardScaler
from scipy.stats import f_oneway
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

data = pd.read_csv("/content/Processed_DataSet_Min-Max_AI.csv")
data.dropna(subset=['sfdm2'], inplace=True)
data.drop(['edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'death', 'hospdead','id'],axis=1, inplace=True)

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
data[columns_to_encode] = data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

In [ ]:
target = data['sfdm2']
x = data.iloc[:, :-1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# 6. Predicting New Instances based on cleated Classification Model
y_predicted = classifier.predict(X_test)

# 7. Evaluation Classification
print(confusion_matrix(y_predicted, y_test))
print(classification_report(y_predicted, y_test))

In [ ]:
# Convert target labels to one-hot encoding for neural network
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.to_numpy().reshape(-1, 1))


# Build a simple neural network
model = Sequential()
model.add(Dense(16, input_dim=37, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model Architecture:", model.summary())
print("        ")


model.fit(X_train, y_train_encoded, epochs=50, batch_size=8, verbose=0)

# Evaluate the neural network on the test set
_, accuracy_nn = model.evaluate(X_test, y_test_encoded)
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")

In [ ]:
y_pred_ensemble = np.argmax(model.predict(X_test) + classifier.predict_proba(X_test), axis=1)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

In [ ]:
print(f'Accuracy of Logical Regression Classifier: {accuracy_rf:.2f}')
print("        ")
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')
class_report_normalized = classification_report(y_test, y_pred_ensemble, zero_division=1)
print("Classification Report (Normalized Dataset):")
print(class_report_normalized)

conf_matrix_normalized = confusion_matrix(y_test, y_pred_ensemble)
print("Confusion Matrix (Normalized Dataset):")
print(conf_matrix_normalized)

###Z-Score NORMALIZED DATASET

In [ ]:
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("/content/Processed_DataSet_Z-score_AI_.csv")
data.dropna(subset=['sfdm2'], inplace=True)
data.drop(['edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'death', 'hospdead','id'],axis=1, inplace=True)

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
data[columns_to_encode] = data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

target = data['sfdm2']
x = data.iloc[:, :-1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=10000)
classifier.fit(X_train, y_train) #Classification Model

# 6. Predicting New Instances based on cleated Classification Model
y_predicted = classifier.predict(X_test)

# 7. Evaluation Classification
print(confusion_matrix(y_predicted, y_test))
print(classification_report(y_predicted, y_test))

In [ ]:
# Convert target labels to one-hot encoding for neural network
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.to_numpy().reshape(-1, 1))


# Build a simple neural network
model = Sequential()
model.add(Dense(16, input_dim=37, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model Architecture:", model.summary())
print("        ")


model.fit(X_train, y_train_encoded, epochs=50, batch_size=8, verbose=0)

# Evaluate the neural network on the test set
_, accuracy_nn = model.evaluate(X_test, y_test_encoded)
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")

In [ ]:
y_pred_ensemble = np.argmax(model.predict(X_test) + clf.predict_proba(X_test), axis=1)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

In [ ]:
print(f'Accuracy of Logical Regression Classifier: {accuracy_rf:.2f}')
print("        ")
print(f'Accuracy of Neural Network: {accuracy_nn:.2f}')
print("        ")
print(f'Accuracy of Ensemble Model: {accuracy_ensemble:.2f}')

class_report_normalized = classification_report(y_test, y_pred_ensemble, zero_division=1)
print("Classification Report (Normalized Dataset):")
print(class_report_normalized)

conf_matrix_normalized = confusion_matrix(y_test, y_pred_ensemble)
print("Confusion Matrix (Normalized Dataset):")
print(conf_matrix_normalized)

# **SFDM2 MODEL: MLP**

###MIN-MAX NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Load normalized dataset
data = pd.read_csv("/content/Processed_DataSet_Min-Max_AI.csv")
data.dropna(subset=['sfdm2'], inplace=True)
data.drop(['edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'death', 'hospdead', 'income'], axis=1, inplace=True)
label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']
data[columns_to_encode] = data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

target = data['sfdm2']
x = data.iloc[:, :-1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.2, random_state=42)


mlp_classifier_normalized = MLPClassifier(
    activation='tanh',
    solver='sgd',
    learning_rate='invscaling',
    random_state=42,
    max_iter=500
)

# Fit the model
mlp_classifier_normalized.fit(X_train, y_train)


y_pred_normalized = mlp_classifier_normalized.predict(X_test)

# Evaluate the model
accuracy_normalized = accuracy_score(y_test, y_pred_normalized)
print("Accuracy (Normalized Dataset):", accuracy_normalized)


class_report_normalized = classification_report(y_test, y_pred_normalized, zero_division=1)
print("Classification Report (Normalized Dataset):")
print(class_report_normalized)


conf_matrix_normalized = confusion_matrix(y_test, y_pred_normalized)
print("Confusion Matrix (Normalized Dataset):")
print(conf_matrix_normalized)


###Z-Score NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Load normalized dataset
data = pd.read_csv("/content/Processed_DataSet_Z-score_AI_.csv")
data.dropna(subset=['sfdm2'], inplace=True)
data.drop(['edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'death', 'hospdead', 'income'], axis=1, inplace=True)
label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']
data[columns_to_encode] = data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

target = data['sfdm2']
x = data.iloc[:, :-1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, target, test_size=0.3, random_state=37)


mlp_classifier_normalized = MLPClassifier(
    activation='tanh',
    solver='sgd',
    learning_rate='invscaling',
    random_state=42,
    max_iter=500
)


mlp_classifier_normalized.fit(X_train, y_train)

# Makeing predictions on the test set
y_pred_normalized = mlp_classifier_normalized.predict(X_test)

# Evaluate the model
accuracy_normalized = accuracy_score(y_test, y_pred_normalized)
print("Accuracy (Normalized Dataset):", accuracy_normalized)


class_report_normalized = classification_report(y_test, y_pred_normalized, zero_division=1)
print("Classification Report (Normalized Dataset):")
print(class_report_normalized)


conf_matrix_normalized = confusion_matrix(y_test, y_pred_normalized)
print("Confusion Matrix (Normalized Dataset):")
print(conf_matrix_normalized)


# **TARGET 2 (DEATH)**

# **Death MODEL: RANDOM FOREST MODEL**


###MIN-MAX NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset
data = pd.read_csv("/content/Processed_DataSet_Min-Max_AI.csv")

# Sampling the dataset
instances_per_target = 10000
sampled_data = data.groupby('death', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))


label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

# Columns to drop
columns_to_drop = ['death', "hospdead", 'edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'id', 'sfdm2']


features = sampled_data.drop(columns=columns_to_drop)
target = sampled_data['death']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=60)

clf = RandomForestClassifier(n_estimators=10000, random_state=60)

# Fit the classifier to the training data
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy:.2f}")


class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


###Z-Score NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset
data = pd.read_csv('/content/Processed_DataSet_Z-score_AI_.csv')

# Sampling the dataset
instances_per_target = 10000
sampled_data = data.groupby('death', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

# Label encoding for categorical columns
label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))

# Columns to drop
columns_to_drop = ['death', "hospdead", 'edu', 'charges', 'totcst', 'totmcst', 'avtisst', 'id', 'sfdm2']


features = sampled_data.drop(columns=columns_to_drop)
target = sampled_data['death']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=60)

# Initialize the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=60)

# Fit the classifier to the training data
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy:.2f}")


class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


#**Death MODEL: DECISION TREE**


###MIN-MAX NORMALIZED DATASET


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Load dataset
data = pd.read_csv('Processed_DataSet_Min-Max_AI.csv')

# Columns to drop
columns_to_drop = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']

# Drop specified columns
data_filtered = data.drop(columns=columns_to_drop)

X = data_filtered.drop(['death'], axis=1)
y = data_filtered['death']

label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

imputer = SimpleImputer(strategy='mean')


X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)


X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


dt_classifier = DecisionTreeClassifier(random_state=42)

# Fit the model
dt_classifier.fit(X_train_imputed, y_train)

y_pred = dt_classifier.predict(X_test_imputed)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)


conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


###Z-Score NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Load dataset
data = pd.read_csv('/content/Processed_DataSet_Z-score_AI_.csv')

# Columns to drop
columns_to_drop = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']

# Drop specified columns
data_filtered = data.drop(columns=columns_to_drop)


X = data_filtered.drop(['death'], axis=1)
y = data_filtered['death']

# Converting categorical variables to numerical using Label Encoding
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


imputer = SimpleImputer(strategy='mean')


X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)


X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


dt_classifier = DecisionTreeClassifier(random_state=42)

# Fit the model
dt_classifier.fit(X_train_imputed, y_train)


y_pred = dt_classifier.predict(X_test_imputed)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)


conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


#**Death MODEL:SVM**

###MIN-MAX NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('/content/Processed_DataSet_Min-Max_AI.csv')

# Columns to drop
columns_to_drop = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']

# Drop specified columns
data_filtered = data.drop(columns=columns_to_drop)


X = data_filtered.drop(['death'], axis=1)
y = data_filtered['death']

# Convert categorical variables to numerical using Label Encoding
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

imputer = SimpleImputer(strategy='mean')


X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)


X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=X_train_imputed.columns)


X_test_scaled = pd.DataFrame(scaler.transform(X_test_imputed), columns=X_test_imputed.columns)

# Initialize an SVM Classifier
svm_classifier = SVC(random_state=42)


svm_classifier.fit(X_train_scaled, y_train)


y_pred = svm_classifier.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate and print the classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)


conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


###Z-Score NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('/content/Processed_DataSet_Z-score_AI_.csv')

# Columns to drop
columns_to_drop = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca', 'sfdm2']

# Drop specified columns
data_filtered = data.drop(columns=columns_to_drop)

X = data_filtered.drop(['death'], axis=1)
y = data_filtered['death']

# Converting categorical variables to numerical using Label Encoding
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Splitin the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


imputer = SimpleImputer(strategy='mean')


X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)


X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


scaler = StandardScaler()


X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=X_train_imputed.columns)


X_test_scaled = pd.DataFrame(scaler.transform(X_test_imputed), columns=X_test_imputed.columns)

# Initialize an SVM Classifier
svm_classifier = SVC(random_state=42)

# Fit the model
svm_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)


conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


#**Target 3 (HOSPITALIZED DEATH)**

#**Hospitalized Death MODEL : Logistic Regression**

###MIN-MAX NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


data = pd.read_csv('/content/Processed_DataSet_Min-Max_AI.csv')

instances_per_target = 1000
sampled_data = data.groupby('hospdead', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


X_hospdead_lr = sampled_data.drop(['edu', 'sfdm2', 'death', 'id', 'hospdead'], axis=1)
y_hospdead_lr = sampled_data['hospdead']

# Split into training and testing sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_hospdead_lr, y_hospdead_lr, test_size=0.2, random_state=42)

# Model Training for Logistic Regression
lr_model = LogisticRegression(max_iter = 10000)
lr_model.fit(X_train_lr, y_train_lr)

# Model Evaluation
y_pred_lr = lr_model.predict(X_test_lr)

accuracy_lr = accuracy_score(y_test_lr, y_pred_lr)
classification_report_lr = classification_report(y_test_lr, y_pred_lr)

print(f"Logistic Regression Accuracy: {accuracy_lr}")
print("Classification Report:")
print(classification_report_lr)


###Z-Score NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


data = pd.read_csv('/content/Processed_DataSet_Z-score_AI_.csv')

instances_per_target = 1000
sampled_data = data.groupby('hospdead', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


X_hospdead_lr = sampled_data.drop(['edu', 'sfdm2', 'death', 'id', 'hospdead'], axis=1)
y_hospdead_lr = sampled_data['hospdead']

# Split into training and testing sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_hospdead_lr, y_hospdead_lr, test_size=0.2, random_state=42)

# Model Training for Logistic Regression
lr_model = LogisticRegression(max_iter = 10000)
lr_model.fit(X_train_lr, y_train_lr)

# Model Evaluation
y_pred_lr = lr_model.predict(X_test_lr)

accuracy_lr = accuracy_score(y_test_lr, y_pred_lr)
classification_report_lr = classification_report(y_test_lr, y_pred_lr)

print(f"Logistic Regression Accuracy: {accuracy_lr}")
print("Classification Report:")
print(classification_report_lr)


#**Hospitalized Death MODEL : DECISION TREE**

###MIN-MAX NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


data = pd.read_csv('/content/Processed_DataSet_Min-Max_AI.csv')

instances_per_target = 1000
sampled_data = data.groupby('hospdead', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


X = sampled_data.drop(['edu', 'sfdm2', 'death', 'id','hospdead'], axis=1)
y = sampled_data['hospdead']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

classifier = DecisionTreeClassifier(criterion='gini')
classifier.fit(x_train, y_train)

#predict the labels of clusters.
y_predicted = classifier.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))

###Z-Score NORMALIZED DATASET

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


data = pd.read_csv('/content/Processed_DataSet_Z-score_AI_.csv')

instances_per_target = 1000
sampled_data = data.groupby('hospdead', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


X = sampled_data.drop(['edu', 'sfdm2', 'death', 'id','hospdead'], axis=1)
y = sampled_data['hospdead']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

classifier = DecisionTreeClassifier(criterion='entropy')
classifier.fit(x_train, y_train)

#predict the labels of clusters.
y_predicted = classifier.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))

#**Hospitalized Death MODEL : NAIVE BAYES**

###Z-Score NORMALIZED DATASET

In [67]:
# 1. Importing Libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

data = pd.read_csv('/content/Processed_DataSet_Z-score_AI_.csv')

instances_per_target = 1000
sampled_data = data.groupby('hospdead', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


X = sampled_data.drop(['edu', 'sfdm2', 'death', 'id','hospdead'], axis=1)
y = sampled_data['hospdead']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 5. Creating Classifier Object and Building Classification Model
classifier = GaussianNB()
classifier.fit(x_train, y_train) #Classification Model

# 6. Predicting New Instances based on cleated Classification Model
y_predicted = classifier.predict(x_test)

# 7. Evaluation Classification
print(confusion_matrix(y_predicted, y_test))
print(classification_report(y_predicted, y_test))


[[259  53]
 [ 40 248]]
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       312
           1       0.82      0.86      0.84       288

    accuracy                           0.84       600
   macro avg       0.85      0.85      0.84       600
weighted avg       0.85      0.84      0.85       600



###MIN-MAX NORMALIZED DATASET

In [ ]:
# 1. Importing Libraries
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

data = pd.read_csv('/content/Processed_DataSet_Min-Max_AI.csv')

instances_per_target = 1000
sampled_data = data.groupby('hospdead', group_keys=False).apply(lambda x: x.sample(min(len(x), instances_per_target)))

label_encoder = LabelEncoder()
columns_to_encode = ['sex', 'dzgroup', 'dzclass', 'race', 'dnr', 'ca','sfdm2']
sampled_data[columns_to_encode] = sampled_data[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


X = sampled_data.drop(['edu', 'sfdm2', 'death', 'id','hospdead'], axis=1)
y = sampled_data['hospdead']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 5. Creating Classifier Object and Building Classification Model
classifier = GaussianNB()
classifier.fit(x_train, y_train) #Classification Model

# 6. Predicting New Instances based on cleated Classification Model
y_predicted = classifier.predict(x_test)

# 7. Evaluation Classification
print(confusion_matrix(y_predicted, y_test))
print(classification_report(y_predicted, y_test))
